In [1]:
import os
import random
import datetime
import numpy as np
import scipy.io as spio

def phi(inVolt,threshold):
    currArr = inVolt/threshold
    return np.where(currArr>1,1,currArr)

def galvesLocherbachSimulator(unitCount,connGraph,remain,thresh,timeSteps):
    #Unit Count, Connection Graph, and timeSteps are all known
    random.seed(2022) #Pick a RNG seed, does not matter which one, just make sure it consistent across all runs
    outputSimmedSpikeTrain = 0
    sampV = np.random.randint(0,thresh,size=(unitCount,))
    for t in range(timeSteps):
        spikeProb = phi(sampV,thresh)
        randArr = np.random.uniform(0,thresh*3,size=(unitCount,))
        trueSpike = np.reshape(np.where(spikeProb>randArr,1,0),(len(spikeProb),1)).astype(int)
        if np.shape(outputSimmedSpikeTrain) == ():
            outputSimmedSpikeTrain = trueSpike
        else:
            outputSimmedSpikeTrain = np.concatenate((outputSimmedSpikeTrain,trueSpike),axis=1)
        voltSpike = np.where(spikeProb>randArr,0,spikeProb)
        voltSpike = voltSpike*remain
        sampV = voltSpike+np.sum(voltSpike*connGraph)
    return outputSimmedSpikeTrain.astype(int)

def generateFakeConnectionGraph(inputUnitCount):
    rawArray = np.random.rand(inputUnitCount,inputUnitCount)
    lowerTriangleInds = np.tril_indices(inputUnitCount)
    rawArray[lowerTriangleInds] = 0
    return rawArray+np.transpose(rawArray)

def getTrueValue(inputFile):
    imat = spio.loadmat(inputFile)
    return imat.get('spaSpike')

def generateNullAccuracy(spikeTrainRefFile):
    trueSpikeTrain = getTrueValue(spikeTrainRefFile)
    unitCount = np.shape(trueSpikeTrain)[0]
    nullAccuracy = []
    for i in range(1000):
        falseConns = generateFakeConnectionGraph(unitCount)
        falseTrain = galvesLocherbachSimulator(unitCount,falseConns,1,18.0475,100)
        currAcc = np.sum(np.abs(trueSpikeTrain[:,4500:4600]-falseTrain))/(np.shape(falseTrain)[0]*100)
        nullAccuracy.append(1-currAcc)
    return nullAccuracy

trueFile = ''
saveFile = ''
dSave = {'nullSpikeTrainAcc':generateNullAccuracy(trueFile)}
var = spio.savemat(saveFile,dSave)
print('Done')


Done
